Thomas Dougherty

Probability and Statistics for Computer Science


Introduction

    To Do:

    Figure out script to iterate through and cleanup all CSVs

    Store data from all CSVs

    Aggregate data from *all* CSVs into one dataframe
    

### March 2018

In [28]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from helper_functions import *

#read CSV for March 2018
services = pd.read_csv('data\\2018_03.csv')
weather = pd.read_csv('data\\weather.csv')
#status column is not needed so that will be dropped
services.drop(labels=['status'], axis=1, inplace=True)
# Amtrak rows are dropped because OTP of Amtrak was not tracked in the dataset
services.drop(services[services['type'] == 'Amtrak'].index, inplace=True)
services = services.astype({'date' : 'datetime64[ns]',
                'train_id' : 'category',
                'stop_sequence' : 'float16',
                'from' : 'category',
                'from_id': 'category',
                'to' : 'category',
                'to_id': 'category',
                'scheduled_time' : 'datetime64',
                'actual_time' : 'datetime64',
                'delay_minutes' : 'float16',
                'line' : 'category',
                'type' : 'category'})


services.dtypes
#services.head(10)

date              datetime64[ns]
train_id                category
stop_sequence            float16
from                    category
from_id                 category
to                      category
to_id                   category
scheduled_time    datetime64[ns]
actual_time       datetime64[ns]
delay_minutes            float16
line                    category
type                    category
dtype: object

In [29]:
 #create new dataframe with data broken down by line
# function to create df. figure out how to import pandas into helper file???
def create_new_dataframe(list_njt_lines, list_max_delays, list_avg_delays, list_dates):
    df = pd.DataFrame({'Longest Delay (minutes)': list_max_delays.values,
                                   'Average Delay (minutes)': list_avg_delays.values,
                                   'Date of Longest Delay' : list_dates.values},
                                    index=list_njt_lines)
    return df

njt_lines=services['line'].unique()
max_delays = pd.Series(helper.get_max_delay(services,njt_lines)).round(3)
avg_delays = pd.Series(helper.get_avg_delay(services,njt_lines)).round(3)
dates = pd.Series(helper.get_delay_date(services,njt_lines))
train_ids = pd.Series(helper.get_train_ids(services,njt_lines))

performance_by_line_march = create_new_dataframe(njt_lines, max_delays, avg_delays, dates)

performance_by_line_march.astype({'Longest Delay (minutes)' :'float16',
                            'Average Delay (minutes)' :'float16',
                            'Date of Longest Delay' : 'datetime64[ns]'
                            })

performance_by_line_march

,Longest Delay (minutes),Average Delay (minutes),Date of Longest Delay
Northeast Corrdr,91.062,4.253,2018-03-12
No Jersey Coast,100.250,3.932,2018-03-01
Main Line,86.000,3.542,2018-03-07
Morristown Line,100.312,3.781,2018-03-02
Gladstone Branch,99.062,3.654,2018-03-02
Raritan Valley,94.000,2.662,2018-03-07
Bergen Co. Line,79.000,2.964,2018-03-07
Atl. City Line,100.188,4.481,2018-03-07
Montclair-Boonton,111.188,3.382,2018-03-07
Princeton Shuttle,4.465,0.096,2018-03-04


#### Analysis:
##### Most of the delays occur between March 1 and March 7. In 2018, two severe winter storms travelled across the eastern United States. One on March 1-2 and March 6-8 just a few days later. The storms brought high wind gusts, precipation, and coastal flooding leading to service delays and cancellations.

###### https://www.njtransit.com/press-releases/nj-transit-implements-severe-weather-rail-schedule-winter-storm

###### https://www.climate.gov/news-features/event-tracker/nor%E2%80%99easters-pummel-us-northeast-late-winter-2018

### November 2018:

In [30]:
#read CSV for March 2018
services_nov = pd.read_csv('data\\2018_11.csv')
#status column is not needed so that will be dropped
services_nov.drop(labels=['status'], axis=1, inplace=True)
# Amtrak rows are dropped because OTP of Amtrak was not tracked in the dataset
services_nov.drop(services_nov[services_nov['type'] == 'Amtrak'].index, inplace=True)
services_nov.drop(services_nov[services_nov['line'] == 'Meadowlands Rail'].index, inplace=True)
services_nov.astype({'date' : 'datetime64[ns]',
                'train_id' : 'category',
                'stop_sequence' : 'float16',
                'from' : 'category',
                'from_id': 'category',
                'to' : 'category',
                'to_id': 'category',
                'scheduled_time' : 'datetime64',
                'actual_time' : 'datetime64',
                'delay_minutes' : 'float16',
                'line' : 'category',
                'type' : 'category'})
services_nov.dtypes
services_nov

njt_lines=services_nov['line'].unique()
max_delays = pd.Series(helper.get_max_delay(services_nov,njt_lines)).round(3)
avg_delays = pd.Series(helper.get_avg_delay(services_nov,njt_lines)).round(3)
dates = pd.Series(helper.get_delay_date(services_nov,njt_lines))
train_ids = pd.Series(helper.get_train_ids(services_nov,njt_lines))

performance_by_line_march = create_new_dataframe(njt_lines, max_delays, avg_delays, dates)

performance_by_line_march.astype({'Longest Delay (minutes)' :'float16',
                            'Average Delay (minutes)' :'float16',
                            'Date of Longest Delay' : 'datetime64[ns]'
                            })

performance_by_line_march

,Longest Delay (minutes),Average Delay (minutes),Date of Longest Delay
No Jersey Coast,91.000,5.541,2018-11-15
Raritan Valley,100.183,6.892,2018-11-23
Bergen Co. Line,75.000,4.995,2018-11-15
Gladstone Branch,117.000,5.173,2018-11-02
Morristown Line,86.000,5.400,2018-11-02
Northeast Corrdr,114.550,4.958,2018-11-20
Main Line,67.117,5.473,2018-11-15
Montclair-Boonton,75.117,5.772,2018-11-13
Pascack Valley,79.000,4.664,2018-11-15


In [31]:
######### weather data 
#####
###
##
weather = weather.astype({'DATE':'datetime64'})

weather['DATE'].apply(pd.Timestamp.month_name)

nov_weather = weather.loc[weather['DATE'].apply(pd.Timestamp.month_name) == 'November']

nov_weather = nov_weather.dropna(axis=1)
nov_weather


C:\Users\tdoug\AppData\Local\Temp\ipykernel_11960\431199858.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nov_weather = nov_weather.dropna(axis=1, inplace=True)
